# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('airflow', case=False)] 
    print(len(df))
    df.to_csv('../dataset/airflow_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

21


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/airflow_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

21


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['Apache Airflow Continous Integration Workflow and Dependency management-Im '
 'thinking of starting to use Apache Airflow for a project and am wondering '
 'how people manage continuous integration and dependencies with airflow. More '
 'specifically Say I have the following set up 3 Airflow servers: dev staging '
 'and production. I have two python DAGS whose source code I want to keep in '
 'seperate repos. The DAGs themselves are simple, basically just use a Python '
 'operator to call main(*args, **kwargs). However the actually code thats run '
 'by main is very large and stretches several files/modules. Each python code '
 'base has different dependencies for example, Dag1 uses Python2.7 '
 'pandas==0.18.1, requests=2.13.0 Dag2 uses Python3.6 pandas==0.20.0 and '
 'Numba==0.27 as well as some cythonized code that needs to be compiled How do '
 'I manage Airflow running these two Dags with completely different '
 'dependencies? Also, how do I manage the continuous integration of 

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['apache', 'airflow', 'continous', 'integration', 'workflow', 'and', 'dependency', 'management', 'im', 'thinking', 'of', 'starting', 'to', 'use', 'apache', 'airflow', 'for', 'project', 'and', 'am', 'wondering', 'how', 'people', 'manage', 'continuous', 'integration', 'and', 'dependencies', 'with', 'airflow', 'more', 'specifically', 'say', 'have', 'the', 'following', 'set', 'up', 'airflow', 'servers', 'dev', 'staging', 'and', 'production', 'have', 'two', 'python', 'dags', 'whose', 'source', 'code', 'want', 'to', 'keep', 'in', 'seperate', 'repos', 'the', 'dags', 'themselves', 'are', 'simple', 'basically', 'just', 'use', 'python', 'operator', 'to', 'call', 'main', 'args', 'kwargs', 'however', 'the', 'actually', 'code', 'thats', 'run', 'by', 'main', 'is', 'very', 'large', 'and', 'stretches', 'several', 'files', 'modules', 'each', 'python', 'code', 'base', 'has', 'different', 'dependencies', 'for', 'example', 'dag', 'uses', 'python', 'pandas', 'requests', 'dag', 'uses', 'python', 'pandas', 

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['continous', 'integration', 'workflow', 'dependency', 'management', 'm', 'think', 'start', 'use', 'airflow', 'project', 'wonder', 'people', 'manage', 'continuous', 'integration', 'dependency', 'airflow', 'specifically', 'say', 'follow', 'set', 'airflow', 'dev', 'staging', 'production', 'python', 'dag', 'source', 'code', 'want', 'keep', 'seperate', 'repos', 'dag', 'simple', 'basically', 'use', 'python', 'operator', 'call', 'main', 'args', 'kwargs', 'however', 'actually', 'code', 's', 'run', 'main', 'large', 'stretch', 'several', 'file', 'module', 'python', 'code', 'base', 'different', 'dependency', 'example', 'use', 'request', 'use', 'well', 'cythonize', 'code', 'need', 'compile', 'manage', 'airflow', 'run', 'dag', 'completely', 'different', 'dependency', 'also', 'manage', 'continuous', 'integration', 'code', 'dag', 'different', 'airflow', 'enivornment', 'dev', 'staging', 'prod', 'get', 'airflow', 'server', 'git', 'pull', 'origin', 'branch', 'hopefully', 'question', 'vague', 'people',

# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 6), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 5), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 4), (15, 4), (16, 2), (17, 3), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 1), (29, 1), (30, 2), (31, 2), (32, 3), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 4), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1),
  ('airflow', 6),
  ('also', 1),
  ('args', 1),
  ('base', 1),
  ('basically', 1),
  ('branch', 1),
  ('call', 1),
  ('code', 5),
  ('compile', 1),
  ('completely', 1),
  ('continous', 1),
  ('continuous', 2),
  ('cythonize', 1),
  ('dag', 4),
  ('dependency', 4),
  ('dev', 2),
  ('different', 3),
  ('enivornment', 1),
  ('example', 1),
  ('file', 1),
  ('follow', 1),
  ('get', 1),
  ('git', 1),
  ('hopefully', 1),
  ('however', 1),
  ('integration', 3),
  ('keep', 1),
  ('kwargs', 1),
  ('large', 1),
  ('m', 2),
  ('main', 2),
  ('manage', 3),
  ('management', 1),
  ('module', 1),
  ('need', 1),
  ('operator', 1),
  ('origin', 1),
  ('people', 2),
  ('problem', 1),
  ('prod', 1),
  ('production', 1),
  ('project', 1),
  ('pull', 1),
  ('python', 3),
  ('question', 1),
  ('repos', 1),
  ('request', 1),
  ('run', 2),
  ('s', 1),
  ('say', 1),
  ('see', 1),
  ('seperate', 1),
  ('server', 1),
  ('set', 1),
  ('several', 1),
  ('simple', 1),
  ('source', 1),
  ('specifi

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

100%|██████████| 540/540 [09:45<00:00,  1.08s/it]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -5.844753813213291

Coherence Score:  0.4149059985207564


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.084534 -0.010024       1        1  41.916776
1      0.047396  0.026128       2        1  16.252600
3      0.036144 -0.045002       3        1  14.172220
2      0.032658  0.007086       4        1  13.840662
4     -0.031664  0.021812       5        1  13.817743, topic_info=          Term       Freq      Total Category  logprob  loglift
294    package  47.000000  47.000000  Default  30.0000  30.0000
428       dist  12.000000  12.000000  Default  29.0000  29.0000
113       site  35.000000  35.000000  Default  28.0000  28.0000
286       line  27.000000  27.000000  Default  27.0000  27.0000
178     import  16.000000  16.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
126       help   1.025692   5.168035   Topic5  -5.6930   0.3621
30           m   1.027724   7.064089   Topic5  -5.6910   0.0515
258    context   1.025798   6.856411   Topic5  -5.6929   0.0795
337       util   1.021946   7.311696   Topic5  -5.6967   0.0114
266  exception   1.021683   7.574794   Topic5  -5.6969  -0.0242

[271 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
108       1  0.317549   access
108       4  0.317549   access
1         1  0.690346  airflow
1         2  0.103552  airflow
1         3  0.086293  airflow
...     ...       ...      ...
339       1  0.571602  version
339       2  0.190534  version
339       4  0.190534  version
245       1  0.328738      web
245       2  0.657475      web

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3, 5])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.044*"file" + 0.043*"airflow" + 0.038*"package" + 0.036*"site" + '
  '0.021*"line" + 0.019*"users_karthikv" + 0.014*"local" + 0.012*"sqlalchemy" '
  '+ 0.011*"error" + 0.010*"import"'),
 (1,
  '0.018*"airflow" + 0.014*"use" + 0.010*"server" + 0.009*"key" + '
  '0.008*"connection" + 0.007*"error" + 0.006*"dag" + 0.006*"code" + '
  '0.006*"httpd" + 0.006*"conf"'),
 (2,
  '0.014*"airflow" + 0.011*"user" + 0.010*"create" + 0.010*"error" + '
  '0.009*"rd" + 0.008*"find" + 0.007*"file" + 0.007*"redshift" + '
  '0.007*"parquet" + 0.007*"load"'),
 (3,
  '0.020*"import" + 0.019*"operator" + 0.016*"airflow" + 0.012*"fail" + '
  '0.011*"plugin" + 0.009*"file" + 0.009*"dataflow" + 0.008*"pipeline" + '
  '0.007*"error" + 0.007*"dag"'),
 (4,
  '0.031*"package" + 0.031*"dist" + 0.029*"file" + 0.022*"line" + '
  '0.013*"sqlalchemy" + 0.013*"sqlalchemy_orm" + 0.011*"self" + '
  '0.010*"execute" + 0.010*"session" + 0.009*"airflow"')]


In [12]:
num_topics = 5
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicAirflow.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  45015116  Apache Airflow Continous Integration Workflow ...   
1  45450618  Connect to Teradata Using Airflow JDBC Connection   
2  48164745  AWS EC2 + Apache Airflow. How to connect to ad...   
3  48986732          Airflow: Creating a DAG in airflow via UI   
4  51558313  What is the difference between min_file_proces...   

                                                Body  RatingsSentiCR  \
0  I'm thinking of starting to use Apache Airflow...               1   
1  I'm trying to execute a SqlSensor task in Airf...              -1   
2  I installed airflow and started it on EC2 Ubun...              -1   
3  Airflow veterans please help,\n\nI was looking...              -1   
4  We are using Airflow v 1.9.0. We have 100+ dag...               0   

   RatingsGPT35  RatingsGPTFineTuned  \
0             1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         

In [ ]:
3   